In [ ]:
import requests
import pandas as pd
import datetime
import json

## Historical Data Scraping from Bitstamp

In [ ]:
market_symbol = "btcusd"
url = f"https://www.bitstamp.net/api/v2/ohlc/{market_symbol}/"
start = "2024-08-01"
end = "2024-08-02"
dates = pd.date_range(start,end,freq="1h")
# 10**9 => the output is in nanoseconds
# int => the output is in float
dates = [int(x.value/10**9) for x in list(dates)]
master_data = []

# iterate over pairs of adjacent elements in list
for first, last in zip(dates, dates[1:]): 
    params = {
    "step":60,
    "limit":1000,
    "start": first,
    "end": last,
    }
    data = requests.get(url, params=params)
    data = data.json()["data"]["ohlc"]
    master_data += data

df = pd.DataFrame(master_data)
df = df.drop_duplicates()

df["timestamp"] = df["timestamp"].astype(int)
df = df.sort_values(by="timestamp")
# make sure there is no extra data except from the specified timestamp
df = df[df["timestamp"] >= dates[0]]
df = df[df["timestamp"] < dates[-1]]

df.to_csv("./data/market_data.csv", index=False)
df